# CS189 Spring 2016: Introduction to Machine Learning

## SID : 23274190 / Name: Hye Soo Choi

### Decision Trees for Classification
In this homework, we will implement decision trees and random forests for classification
on 2 datasets: 
 1. the spam data 
 2. a census income dataset 
 
 
to predict whether or not a person makes over 50k in income. 
In lectures, we were given a basic introduction to decision trees and how such trees
are trained. We were also introduced to random forests and boosting algorithms. We have freedom to research different decision tree techniques online. 

In [1]:
import numpy as np
import scipy.io as sio
import pandas as pd
from collections import Counter
from operator import itemgetter
from statistics import mode
from scipy import stats
from sklearn.cross_validation import KFold


In [2]:
spam_rawdata = sio.loadmat('./spam-dataset/spam_data.mat')
census_rawdata = pd.read_csv('./census_data/train_data.csv')

In [67]:
np.random.seed(0)

sp_test = spam_rawdata['test_data']
sp_dataf = spam_rawdata['training_data']
sp_datav = spam_rawdata['training_labels'][0,:]

### Preprocessing : Census data

In [12]:
census_rawdata.values.shape
census_fv = census_rawdata.values[:,0:14]
census_lb = census_rawdata.values[:,14]
census_test = np.array(pd.read_csv('./census_data/test_data.csv').values)

In [17]:
type(census_fv[0,2])

int

Assuming that economic welfare of the native country can be assessed by its GDP per capita, we convert the native country(column 13) into its GDP per capita. Also, we deleted the 3rd column, education, since it is redundant considering the next column 'education number' also indicates education level. We think that the education levels can be treated as ordered categorical values since in general sense, the higher education a person   and this removal would facilitate the overal analysis.

In [13]:
for i in range(14):
    fv = census_fv[:,i]

    if type(fv[0]) == str or type(fv[0]) == np.str_:
        values = list(Counter(fv).keys())
        num = len(values)
        count= [0 for m in range(num)]
        count1 = [0 for m in range(num)]
        for j in range(len(fv)):
            for k in range(num):
                if fv[j] == values[k] :
                    count[k] += 1
                    count1[k] += 1
        prop = np.array([count1[l]/count[l] for l in range(num)])
        def compare_prop(x):
            return prop[x]
        
        new_values = sorted(list(range(num)), key = compare_prop)

        dic = {values[m]: new_values[m] for m in range(num)}
        new_fv = [dic[m] for m in fv]
        census_fv[:,i] = new_fv
        
 
    

### Preprocessing: Spam data
- Over-sample :


Since spam data is somewhat skewed in that the number of spam is much bigger than the number of ham. To handle this data, we will over-sample the given data and therefore achieve a more balanced class distributions by replicating examples of the minority class.

- Transform each element into proportion of the numbers of featured word in the total numbers in email.


In [118]:
def convert_to_zero_one(x):
    1 * (x > 0)
    
def convert_to_prop(x):
    total = sum(x)
    if total == 0:
        total = 1
    return [x[i]/total for i in range(len(x))]

In [161]:
new_feat = []
for i in range(len(sp_datav)):
    new_feat.append([sum(sp_dataf[i,:])])
sp_dataf_ext = np.append(sp_dataf, new_feat, axis = 1)

new_feat = []
for i in range(sp_test.shape[0]):
    new_feat.append([sum(sp_test[i,:])])
sp_test_ext = np.append(sp_test, new_feat, axis = 1)

In [152]:
sp_dataf_prop = []
sp_test_prop = []
for i in range(len(sp_datav)):
    sp_dataf_prop.append(convert_to_prop(sp_dataf[i,:]))
sp_dataf_prop = np.array(sp_dataf_prop)

for i in range(sp_test.shape[0]):
    sp_test_prop.append(convert_to_prop(sp_test[i,:]))
sp_test_prop = np.array(sp_test_prop)


spam_ind = np.array([i for i in range(len(sp_datav)) if sp_datav[i] == 1])
over_ind = np.append(range(len(sp_datav)), spam_ind)
sp_overf_ext = sp_dataf_ext[over_ind, :]
sp_overv = sp_datav[over_ind]


In [163]:
sp_test_ext.shape

(5857, 33)

In [153]:
spam_ind = np.array([i for i in range(len(sp_datav)) if sp_datav[i] == 1])
over_ind = np.append(range(len(sp_datav)), spam_ind)
sp_overf = sp_dataf[over_ind, :]
sp_overv = sp_datav[over_ind]

In [78]:
values =  [1,2,3,4,5,6,6,7,8,9,7,6,5,4,3,2,1,10]
labels = [0,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0]
counter = Counter(values)
uvalues = sorted(counter.keys())
ufreq = [counter[i] for i in uvalues]
pair = [[values[i], labels[i]] for i in range(len(values))]
sort_pair = sorted(pair, key= itemgetter(0))
sort_label = []
count = 0
risk = 0

          
for i in range(len(uvalues)):
    sort_label.append([sort_pair[j][1]  for j in range(count, count + ufreq[i])])
    count = count + ufreq[i]
                
left_labels = []
right_labels = sum(sort_label, [])

In [107]:
non_ind = np.array([i for i in range(len(census_lb)) if census_lb[i] == 1])
over_ind = np.append(range(len(census_lb)), np.append(non_ind, np.append(non_ind, non_ind)))
cs_overf = census_fv[over_ind, :]
cs_overv = census_lb[over_ind]

1250

#### Missing Value

We implement within our decision tree functionality to handle missing feature values based on the current node. More precisely, we infer missing values based on the mode of the feature values of data points sorted to the current
node.

### Code for DecisionTree class

#### Choosing categorical data

We keep the categories as strings. Then we implement functionality in decision trees to determine split rules based
on the subsets of categorical variables that maximizes information gain.

#### stopping condition

We stop growing tree when a node meets any of the following conditions:

 1. its depth of tree is greater than the max depth,
 2. the number of data point in a node is less than 10,
 3. more than 95% of labels in the node are of the same label.

In [154]:

def hist_labels(labels):
    def f(n):
        if n == 1:
            return sum(labels)
        else:
            return len(labels) - sum(labels)
    return f

def powerset(orig, newset):
    if orig == []:
        return [newset]
    else:
        res = []
        for s in powerset(orig[1:], newset + [orig[0]]):
            res.append(s)
        for s in powerset(orig[1:], newset):
            res.append(s)
        return res
    
def entropy(p):
    if p == 0:
        return 0
    else:
        return - p * np.log(p)/np.log(2)

class DecisionTree(object):
    def __init__(self, data, labels, limit):
        self.root = None
        self.values = list(Counter(labels).keys())
        self.data = data
        self.labels = labels
        self.limit = limit
        self.prune_count = 0
        self.node_count = 0 
        self.threshold = 4/6

    
    def impurity(self, left_label_hist, right_label_hist):
        '''A method that takes in the result of a split: two histograms 
        (a histogram is a mapping from label values to their frequencies) 
        that count the frequencies of labels on the "left" and "right" side of that split. 
        The method calculates and outputs a scalar value representing the impurity 
        (i.e. the "badness") of the specified split on the input data.'''
        
        left_freq = [left_label_hist(i) for i in self.values]
        right_freq = [right_label_hist(i) for i in self.values]
        
        left_total = sum(left_freq)
        right_total = sum(right_freq)
        
        left_prob = [i/ left_total for i in left_freq]
        right_prob = [ i/ right_total for i in right_freq]
        
        left_loss = sum([entropy(p) for p in left_prob])
        right_loss = sum([entropy(p) for p in right_prob])
        
        return (left_total * left_loss + right_total * right_loss)/ (left_total + right_total)
    
    
    def gini_impurity(self, left_label_hist, right_label_hist):
        '''A method that takes in the result of a split: two histograms 
        (a histogram is a mapping from label values to their frequencies) 
        that count the frequencies of labels on the "left" and "right" side of that split. 
        The method calculates and outputs a scalar value representing the impurity 
        (i.e. the "badness") of the specified split on the input data.'''
        
        left_freq = [left_label_hist(i) for i in self.values]
        right_freq = [right_label_hist(i) for i in self.values]
        
        left_total = sum(left_freq)
        right_total = sum(right_freq)
        
        left_prob = [i/ left_total for i in left_freq]
        right_prob = [ i/ right_total for i in right_freq]
        
        left_loss = sum([p*(1-p) for p in left_prob])
        right_loss = sum([p*(1-p) for p in right_prob])
        
        return (left_total * left_loss + right_total * right_loss)/ (left_total + right_total)
    
    def segmenter(self, data, labels): 
        '''A method that takes in data and labels. When called, it finds the best split rule 
        for a Node using the impurity measure and input data. There are many different types of 
        segmenters you might implement, each with a different method of choosing a threshold. 
        The usual method is exhaustively trying lots of different threshold values from the data 
        and choosing the combination of split feature and threshold with the lowest impurity value. 
        The final split rule uses the split feature with the lowest impurity value and the threshold chosen by the 
        segmenter. 
        '''
        
        counter = Counter(labels)
        keys = list(counter.keys())
        freq = list(counter.values())
        
        n = len(labels)
        risk = 0

        if freq[0]/n == 1:
            return keys[0]

        
        elif sum(freq) < len(self.labels)/2000:
            if sum(labels) / len(labels) > self.threshold:
                return 1
            else:
                return 0
            
        else:
            num_feat = data.shape[1]

            for i in range(num_feat) :
                feat_values = data[ :,i]
                counter_fvalues = Counter(feat_values)
                fvkeys = list(counter_fvalues.keys())
                fvfreq = list(counter_fvalues.values())
                if len(fvkeys) == 1:
                    continue
                elif type(feat_values[0]) == str or type(feat_values[0]) == np.str_ :
                    temp = self.segmenter_str(feat_values, labels)
                else:
                    temp = self.segmenter_int(feat_values, labels)
                    
                temp_segment = temp[0]
                temp_risk = temp[1]
                if risk == 0:
                    risk = temp_risk
                    segment = [i, temp_segment]
                elif risk > temp_risk:
                    segment = [i, temp_segment]
                    risk = temp_risk
            if risk == 0:
                if sum(labels)/len(labels) > self.threshold:
                    return 1
                else:
                    return 0
            else:
                return segment
            
                
    def segmenter_int(self, values, labels):
        missing_ind = [i for i in range(len(values)) if values[i] == '?']
        nomis_values = [values[j] for j in range(len(values)) if j not in missing_ind]
        mode = np.mean(nomis_values)
        for i in missing_ind:
            values[i] = mode
        
        counter = Counter(values)
        uvalues = sorted(counter.keys())
        ufreq = [counter[i] for i in uvalues]
        pair = [[values[i], labels[i]] for i in range(len(values))]
        sort_pair = sorted(pair, key= itemgetter(0))
        sort_label = []
        count = 0
        risk = 0

          
        for i in range(len(uvalues)):
            sort_label.append([sort_pair[j][1]  for j in range(count, count + ufreq[i])])
            count = count + ufreq[i]
                
        left_labels = []
        right_labels = sum(sort_label, [])
        for i in range(len(uvalues)-1):
            left_labels = left_labels + sort_label[i]
            right_labels = right_labels[ufreq[i]:]
        
            left_histogram = hist_labels(left_labels)
            right_histogram = hist_labels(right_labels)
            temp_risk = self.impurity(left_histogram, right_histogram)
            
            if i == 0 :
                risk = temp_risk
                split = (uvalues[i] + uvalues[i+1])/2
            elif risk > temp_risk:
                risk = temp_risk
                split = (uvalues[i] + uvalues[i+1])/2
            
        def split_rule(x):
            if x == -1:
                return split
            else:
                return x > split
        
        return [split_rule, risk]
                
        
    def segmenter_str(self, values, labels):
        missing_ind = [i for i in range(len(values)) if values[i] == '?']
        nomis_values = [values[j] for j in range(len(values)) if j not in missing_ind]
        mode = max(set(nomis_values), key=nomis_values.count)
        for i in missing_ind:
            values[i] = mode

            
        counter = Counter(values)
        uvalues = sorted(counter.keys())
        ufreq = [counter[i] for i in uvalues]
        pair = [[values[i], labels[i]] for i in range(len(values))]
        sort_pair = sorted(pair, key= itemgetter(0))
        sort_label = []
        count = 0
        
        for i in range(len(uvalues)):
            sort_label.append([sort_pair[j][1]  for j in range(count, count + ufreq[i])])
            count = count + ufreq[i]
            
        count = 0      
        left_labels = []
        right_labels = []
        risk = 0
        power_values = [l for l in powerset([j for j in range(len(uvalues))],[]) 
                        if len(l) <= len(uvalues)/2  and len(l) > 0 ]
        
        
        for indice in power_values:
      
            left_labels = sum([sort_label[h] for h in indice],[])
            right_labels = sum([sort_label[h] for h in range(len(uvalues)) if h not in indice],[])
        
            left_histogram = hist_labels(left_labels)
            right_histogram = hist_labels(right_labels)
            temp_risk = self.impurity(left_histogram, right_histogram)

            if risk == 0 :
                risk = temp_risk
                split = [uvalues[i] for i in indice]
            elif risk > temp_risk:
                risk = temp_risk
                split = [uvalues[i] for i in indice]
        def split_rule(x):
            if x == -1:
                return split
            else:
                return x in split
        
        return [split_rule, risk]
    
    
    
    def train(self, data, labels):
        

        def construct_node(cdata,clabels, depth):


            
            counter = Counter(clabels)
            ulabel = list(counter.keys())
            self.node_count += 1
            
            if len(ulabel)<=1:
                n = len(clabels)
                    
                return Node(label = ulabel[0], impurity = 0, num = n)
            else:
                segment = self.segmenter(cdata, clabels)
           
            
            if type(segment) != list:
                n = len(clabels)
                if segment == 0:
                    imp = len([i for i in range(len(clabels)) if clabels[i] != segment])/ n
                else:
                    imp =  len([i for i in range(len(clabels)) if clabels[i] != segment])/ n
                
                return Node(label = segment, impurity = imp, num = n)
            
            elif depth > self.limit:
                p = sum(clabels)/len(clabels)
                n = len(clabels)
                
                if p > self.threshold:
                    temp_label = 1
                    imp = ( len([i for i in range(len(clabels)) if clabels[i] != temp_label]))/ n
                else:
                    temp_label = 0
                    imp = len([i for i in range(len(clabels)) if clabels[i] != temp_label])/ n
                
                return Node(label = temp_label, impurity = imp, num=n)

            else: 
                split_rule = segment[1]
                split_fv = cdata[: , segment[0]]
                missing_ind = [i for i in range(len(split_fv)) if split_fv[i] == '?']
                nomis_values = [split_fv[j] for j in range(len(split_fv)) if j not in missing_ind]
                mode = np.mean(nomis_values)
                for i in missing_ind:
                    split_fv[i] = mode


                split_cond = np.array([split_rule(i) for i in split_fv])
                left_data = cdata[split_cond,:]
                left_labels = clabels[split_cond]
                right_data = cdata[~ split_cond,:]
                right_labels = clabels[~ split_cond]
                
                p = sum(clabels) / len(clabels)
                
                n = len(clabels)
                if p > self.threshold :
                    temp_label = 1
                    imp = (len([i for i in range(len(clabels)) if clabels[i] != temp_label]))/ n
                else:
                    temp_label = 0
                    imp = len([i for i in range(len(clabels)) if clabels[i] != temp_label])/ n
                return Node(split_rule= segment, left = construct_node(left_data, left_labels, depth+1),
                            right = construct_node(right_data, right_labels, depth+1), impurity = imp, num = n,
                           temp_label = temp_label)
            
        self.root = construct_node(data, labels, 0)
        
    def predict(self, data):
        n = data.shape[0]
        res = [-1 for i in range(n)]
        def test(cdata, current, ind):
            
            if len(ind) == 0:
                return
                
            elif current.label != None:
                for i in ind:
                    res[i] = current.label
            else: 
                current_rule = current.split_rule[1]
                current_index = current.split_rule[0]
                fv = cdata[:, current_index]
                missing_ind = [i for i in range(len(fv)) if fv[i] == '?']
                nomis_values = [fv[j] for j in range(len(fv)) if j not in missing_ind]
                mode = max(set(nomis_values), key=nomis_values.count)
                for i in missing_ind:
                    fv[i] = mode          
                
                split_cond = np.array([current_rule(i) for i in fv])
                left_data = cdata[split_cond, :]
                right_data = cdata[~split_cond, :]
                left_ind = ind[split_cond]
                right_ind = ind[~split_cond]
                if len(left_ind) > 0:
                    test(left_data, current.left, left_ind)
                if len(right_ind) > 0:
                    test(right_data, current.right, right_ind)
        test(data, self.root, np.array(list(range(n))))
        
        return res
    
    def prune(self, alpha):
        

        def prune_node(current):
            if current.left.label != None and current.right.label != None:
     
                
                
                post_error = (current.left.num * current.left.impurity 
                              + current.right.num * current.right.impurity)/ current.num

                if current.impurity < post_error + alpha:
                    current.label =  current.temp_label
                    current.left = None
                    current.right = None
                    self.prune_count += 1
                    self.node_count -= 1
                    
                    
        def prune_tree(current):
            if current.num == 0:
                return
            elif current.label == None:       
                prune_tree(current.left)
                prune_tree(current.right)
                prune_node(current)
                

        prune_tree(self.root)
        print(self.prune_count)
        print(self.node_count)

                  
            
        
            

class Node(object):
    def __init__(self, split_rule = None, left = None, right = None, label = None, impurity = None, 
                 temp_label = None, num = 0):
        self.split_rule = split_rule
        self.left = left
        self.right = right
        self.label = label
        self.impurity = impurity
        self.temp_label = temp_label
        self.num = num

        
    

### Cross-validation to determine the hyperparameter for pruning

Starting from each leaf, if the decrease in impurity, which we gain as a result of having the parent node splitted, is less than $\alpha$, then we choose to prune the subtree that has the parent node as its root. To determine best value of $\alpha $, we use 10-fold cross-validation.

In [155]:
kf = KFold(len(sp_overv), n_folds=5)
def cv_pruning(alpha):
    test_error = 0 

    for train_index, test_index in kf:
    
        X_train, X_test = sp_overf_ext[train_index, : ], sp_overf_ext[test_index, :]
        y_train, y_test = sp_overv[train_index], sp_overv[test_index]
        sp_tree = DecisionTree(X_train, y_train, 50)
        sp_tree.train(X_train, y_train)
        sp_tree.prune(alpha)
        sp_predcv =sp_tree.predict(X_test)
        count = 0
        for j in range(len(y_test)):
            if y_test[j] != sp_predcv[j]:
                count += 1
        test_error += count/len(y_test)
    
    return test_error/5
                
        

In [156]:
cv_pruning(1e-2)

117
672
116
591
135
556
132
597
123
680


0.2291581175704283

In [157]:
cv_pruning(5e-2)

122
667
138
569
138
553
137
592
136
667


0.2335044837131996

In [52]:
cv_pruning(1e-1)

213
738
154
653
187
612
212
615
182
715


0.22766189938738496

#### Cross-validation to determine the hyperparameter for pruning with census data

In [34]:
kf = KFold(len(census_lb), n_folds=5)
def cv_pruning_cs(alpha):
    test_error = 0 

    for train_index, test_index in kf:
    
        X_train, X_test = census_fv[train_index, : ], census_fv[test_index, :]
        y_train, y_test = census_lb[train_index], census_lb[test_index]
        cs_tree = DecisionTree(X_train, y_train, 10)
        cs_tree.train(X_train, y_train)
        cs_tree.prune(alpha)
        cs_predcv =cs_tree.predict(X_test)
        count = 0
        for j in range(len(y_test)):
            if y_test[j] != cs_predcv[j]:
                count += 1
        test_error += count/len(y_test)
    
    return test_error/5
                

In [ ]:
cv_pruning_cs(1e-2)

As a result of cross-validation, we learn the best $\alpha$ value is $10^{-1}$ for spam data and for census data

### Submission to Kaggle
With the best $\alpha$ value, we trained a single decision tree and test the model on the test data. This achieved for spam data and for census data.

In [164]:
sp_final_tree = DecisionTree(sp_overf_ext, sp_overv, 50)
sp_final_tree.train(sp_overf_ext, sp_overv)
sp_final_tree.prune(1e-2)
dt_pred = sp_final_tree.predict(sp_test_ext)

120
653


In [144]:
sp_test_prop.shape

(5857, 32)

In [166]:
dt_pred = sp_final_tree.predict(sp_test_ext)
sum(dt_pred)

1715

In [167]:
dt_pred_txt = np.asarray([[i+1, dt_pred[i]] for i in np.arange(5857)])
np.savetxt('dt_sp_pred4.csv', dt_pred_txt, fmt = '%1.u' , delimiter = ',', header = 'Id,Category',comments='')

In [ ]:
cs_final_tree = DecisionTree(census_fv, census_lb, 500)
cs_final_tree.train(census_fv, census_lb)
cs_final_tree.prune(1e-2)
cs_pred = cs_final_tree.predict(census_test)

In [122]:
cs_pred_txt = np.asarray([[i+1, cs_pred[i]] for i in np.arange(5857)])
np.savetxt('cs_pred.csv', cs_pred_txt, fmt = '%1.u' , delimiter = ',', header = 'Id,Category',comments='')

In [66]:
print(sp_final_tree.root.split_rule[1](-1))
print(sp_final_tree.root.split_rule[0])
print(sp_final_tree.root.right.split_rule[1](-1))
print(sp_final_tree.root.right.split_rule[0])
print(sp_final_tree.root.right.right.split_rule[1](-1))
print(sp_final_tree.root.right.right.split_rule[0])


10.5
13
9.0
13
10.0
14


#### Split conditions:

For spam data,

1. 9th feature > 0.0
2. 3rd feature > 0.0
3. 23rd feature > 0.0

were the first three conditions that splits the data.

#### Bagging

In [39]:
dt_pred_txt = np.asarray([[i+1, dt_pred[i]] for i in np.arange(5857)])
np.savetxt('dt_sp_pred.csv', dt_pred_txt, fmt = '%1.u' , delimiter = ',', header = 'Id,Category',comments='')

In [163]:
a = np.array([1,2,3])
b = np.array([1,2,2,0,0])
a[b]

array([2, 3, 3, 1, 1])

In [101]:
sp_test = [1*(pred[i] > 50) for i in range(5857)]
sp_pred = np.asarray([[i+1, sp_test[i]] for i in np.arange(5857)])
np.savetxt('sp_test2.csv', sp_pred, fmt = '%1.u' , delimiter = ',', header = 'Id,Category',comments='')

In [24]:
test2 = DecisionTree(census_fv, census_lb, 100)

In [118]:
clabels = np.array([1,1,1,1,1,1,4,4,5,2,2,2,2,2,2])
stats.mode(clabels)[0]

array([1])

In [35]:
cs_pred =[0 for i in range(16118)]
for i in range(1):
    id0 = np.random.choice(census_fv.shape[0], 3000, replace = True)
    test2.train(census_fv[id0, :], census_lb[id0])
    pred0 = test2.predict(census_test)
    cs_pred = [(cs_pred[i] + pred0[i]) for i in range(16118)]

In [36]:
print(test2.root.split_rule[1](-1))
print(test2.root.split_rule[0])
print(test2.root.left.split_rule[0])

print(test2.root.left.split_rule[1](-1))
print(test2.root.left.left.split_rule[0])

print(test2.root.left.left.split_rule[1](-1))

 def prune(self, data, labels, alpha):

        def prune_node(current, cdata, clabels):
            if current.left.label != None and current.right.label != None:
     
                current_rule = current.split_rule[1]
                current_index = current.split_rule[0]
                fv = cdata[:, current_index]
                missing_ind = [i for i in range(len(fv)) if fv[i] == '?']
                nomis_values = [fv[j] for j in range(len(fv)) if j not in missing_ind]
                mode = max(set(nomis_values), key=nomis_values.count)
                for i in missing_ind:
                    fv[i] = mode          
                
                split_cond = np.array([current_rule(i) for i in fv])
                left_data = cdata[split_cond, :]
                right_data = cdata[~split_cond, :]
                left_labels = clabels[split_cond]
                right_labels = clabels[~split_cond]
                left_hist = hist_labels(left_labels)
                right_hist = hist_labels(right_labels)
                
                

                if current_error < post_error:
                    current.label = stats.mode(clabels)[0][0]
                    current.left = None
                    current.right = None
                    
        def prune_tree(current, cdata, clabels):
            if len(clabels) == 0:
                return
            elif current.label == None:
                
            
                current_rule = current.split_rule[1]
                current_index = current.split_rule[0]
                fv = cdata[:, current_index]
                missing_ind = [i for i in range(len(fv)) if fv[i] == '?']
                nomis_values = [fv[j] for j in range(len(fv)) if j not in missing_ind]
                mode = max(set(nomis_values), key=nomis_values.count)
                for i in missing_ind:
                    fv[i] = mode          
                
                split_cond = np.array([current_rule(i) for i in fv])
                left_data = cdata[split_cond, :]
                right_data = cdata[~split_cond, :]
                left_labels = clabels[split_cond]
                right_labels = clabels[~split_cond]
                prune_tree(current.left, left_data, left_labels)
                prune_tree(current.right, right_data, right_labels)
                prune_node(current, cdata, clabels)
        
        prune_tree(self.root,data,labels)



17
0
0
18
4
['Married-AF-spouse']


In [37]:
cs_test = [1*(cs_pred[i] > 0) for i in range(16118)]
cs_pred_ = np.asarray([[i+1, cs_test[i]] for i in np.arange(16118)])
np.savetxt('cs_test1.csv', cs_pred_, fmt = '%1.u' , delimiter = ',', header = 'Id,Category',comments='')

#### Kaggle score
This results in Kaggle score 0.6837 for spam data.